In [1]:
# Python Imports
import ipywidgets as widgets
import networkx as nx
import matplotlib
from datetime import timedelta
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# Project Imports
from src.mesh_analysis.readers.file_reader import FileReader
from src.mesh_analysis.tracers.waku_tracer import WakuTracer

In [2]:
from src.mesh_analysis.visualizer.waku_visualizer import add_elapsed_time

tracer = WakuTracer()
reader = FileReader("../../../test/log_test/", tracer)

df = reader.read()
df = add_elapsed_time(df)

In [3]:
df

sender_peer_id receiver_peer_id  \
timestamp                        msg_hash                                   
2024-04-22 14:06:58.001000+00:00 0x1                   A                B   
2024-04-22 14:06:58.002000+00:00 0x1                   B                D   
2024-04-22 14:06:58.003000+00:00 0x1                   A                C   
                                 0x1                   D                E   
2024-04-22 14:06:58.006000+00:00 0x1                   A                F   
2024-04-22 14:06:59.001000+00:00 0x2                   A                B   

                                           elapsed_time  
timestamp                        msg_hash                
2024-04-22 14:06:58.001000+00:00 0x1                  0  
2024-04-22 14:06:58.002000+00:00 0x1                  1  
2024-04-22 14:06:58.003000+00:00 0x1                  2  
                                 0x1                  2  
2024-04-22 14:06:58.006000+00:00 0x1                  5  
2024-04-22 14:06:59.001000+00:00 0x2                  0

In [4]:
%matplotlib
dropdown1 = widgets.SelectMultiple(
                        options=df.index.get_level_values(0).drop_duplicates(),
                        description='Timestamp',
                        disabled=False,
                        layout={'height':'100px', 'width':'100%'})
# todo: modify dropdown2 depending on 1
dropdown2 = widgets.SelectMultiple(
                        options=df.index.get_level_values(1).drop_duplicates(),
                        description='Msg hash',
                        disabled=False,
                        layout={'height':'100px', 'width':'100%'}) 
    
fig = plt.figure(figsize=(10,10))

ax_ = fig.add_subplot(1, 1, 1)
ax_.set_xlim(-3, 3)
ax_.set_ylim(-3, 3)

data_prepared = df.reset_index()
complete_graph = nx.from_pandas_edgelist(data_prepared, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash', 'elapsed'], create_using=nx.DiGraph)
g_pos_layout = nx.kamada_kawai_layout(complete_graph)


def test(y, x, data_):
    # todo maintain limits?
    ax_.clear()
    ax_.set_xlim(-2, 2)
    ax_.set_ylim(-2, 2)
    # 1.
    data_ = data_[data_.index.get_level_values(1) == y[0]]

    # 3. 
    first_timestamp = data_.index.get_level_values(0)[0]
    print(f"first timestamp: {first_timestamp}")
    print(f"selected timestamp: {y[0]}")
    
    df_ = data_.loc[(first_timestamp,y[0]):(x[0], y[0])]
    df_.reset_index(inplace=True)
    
    print(f"selected df \n {df_}")
    selected_graph = nx.from_pandas_edgelist(df_, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash', 'elapsed'], create_using=nx.DiGraph)
    
    # 4. 
    g_pos_layout_selected = {key: g_pos_layout[key] for key in list(selected_graph.nodes)}
    print(f"node positions {g_pos_layout_selected}")
    nx.draw(selected_graph, g_pos_layout_selected, ax=ax_, with_labels=True)
    edge_labels = {} 
    for u, v, data in selected_graph.edges(data=True):
        edge_labels[u, v] = f"{data['elapsed']}ms"
    print(f"edge labels {edge_labels}")
    nx.draw_networkx_edge_labels(selected_graph, g_pos_layout_selected, edge_labels=edge_labels, ax=ax_)
    plt.show()


i_plot = widgets.interactive(test, x=dropdown1, y=dropdown2, data_=widgets.fixed(df))
i_plot


Using matplotlib backend: QtAgg


interactive(children=(SelectMultiple(description='Msg hash', layout=Layout(height='100px', width='100%'), opti…